In [1]:
import numpy as np
import Utils as ut

In [2]:
from sklearn import datasets
# make_classification(n_samples=100, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, 
#                     n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, 
#                     hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)
ds, label = datasets.make_classification(n_samples=300, n_features=30, n_informative=10, n_redundant=0, n_classes=2)

In [3]:
def random_sampling(X, y, n_samples):
    """Extract n_samples from X and y."""
    from sklearn.model_selection import StratifiedShuffleSplit
    
    train_idx, test_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=n_samples).split(X, y))
    return X[train_idx, :], y[train_idx], X[test_idx,:], y[test_idx]

In [4]:
ds1 = np.hstack([ds[:,:5], ds[:,10:15]])
ds1_tr, l1_tr, ds1_ts, l1_ts = random_sampling(ds1, label, n_samples=0.25)

ds2 = np.hstack([ds[:,5:10], ds[:,15:20]])
ds2_tr, l2_tr, ds2_ts, l2_ts = random_sampling(ds2, label, n_samples=0.25)

ds3 = ds[:,20:]
ds3_tr, l3_tr, ds3_ts, l3_ts = random_sampling(ds3, label, n_samples=0.25)

In [5]:
k11 = ut.kernel(ds1_tr, K_type='linear').kernelMatrix(ds1_tr)
k12 = ut.kernel(ds1_tr, K_type='polynomial', param = 3).kernelMatrix(ds1_tr)
k13 = ut.kernel(ds1_tr, K_type='gaussian', param = 0.5).kernelMatrix(ds1_tr)
k21 = ut.kernel(ds2_tr, K_type='linear').kernelMatrix(ds2_tr)
k22 = ut.kernel(ds2_tr, K_type='polynomial', param = 3).kernelMatrix(ds2_tr)
k23 = ut.kernel(ds2_tr, K_type='gaussian', param = 0.5).kernelMatrix(ds2_tr)
k31 = ut.kernel(ds3_tr, K_type='linear').kernelMatrix(ds3_tr)
k32 = ut.kernel(ds3_tr, K_type='polynomial', param = 3).kernelMatrix(ds3_tr)
k33 = ut.kernel(ds3_tr, K_type='gaussian', param = 0.5).kernelMatrix(ds3_tr)

k_list = [k11, k12, k13, k21, k22, k23, k31, k32, k33]

In [6]:
weights = ut.centeredKernelAlignment(k_list, label)

-0.19020099594054482